In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
import plotly.graph_objs as go

In [3]:
print(__version__)

3.5.0


In [4]:
import pandas as pd

In [5]:
zetus = pd.read_csv("https://raw.githubusercontent.com/gautsi/gen-purpose-repo/master/zetus.csv")

In [6]:
zetus["time_"] = pd.to_datetime(zetus.time)

In [7]:
zetus.head()

,time,dur,type,time_
0,01/18/19 10:00,15,R,2019-01-18 10:00:00
1,01/18/19 10:00,10,L,2019-01-18 10:00:00
2,01/18/19 12:15,15,R,2019-01-18 12:15:00
3,01/18/19 13:00,0,S,2019-01-18 13:00:00
4,01/18/19 14:35,10,L,2019-01-18 14:35:00


In [8]:
init_notebook_mode(connected=True)

In [9]:
zetus.dtypes

time             object
dur               int64
type             object
time_    datetime64[ns]
dtype: object

In [10]:
import colorlover as cl

In [11]:
colorscale = cl.scales['5']['qual']['Dark2']

In [12]:
feed_data = [
    go.Bar(
        x = zetus[zetus["type"] == side].sort_values("time_").time_,
        y = 2 * (i - 0.5) * zetus[zetus["type"] == side].sort_values("time_").dur,
        name = side,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i]}) for i, side in enumerate(["L", "R"])
]

output_data = [
    go.Bar(
        x = zetus[zetus["type"] == output].sort_values("time_").time_,
        y = len(zetus[zetus["type"] == output]) * [60],
        name = output,
        base = -30,
        width = 4 * 1000 * 60,
        marker = {"color": colorscale[i + 2]}) for i, output in enumerate(["S", "W"])
]

layout = go.Layout()

fig = go.Figure(data = feed_data + output_data, layout = layout)

iplot(fig)

In [ ]:
plot(fig)